# Suggested solution to Problem 14(a)

In [28]:
using Oscar

## Define the polynomial we want to investigate

In [29]:
R, x = PolynomialRing(QQ,"x" => 1:2)
f =  x[2] - 4*x[1]*x[2]^3 + x[1]^2*x[2]^4 + 8*x[1]^3*x[2]^4

8*x_{1}^3*x_{2}^4 + x_{1}^2*x_{2}^4 - 4*x_{1}*x_{2}^3 + x_{2}

## Compute the vertices of the Newton polytope

This is how to get a list of all vertices:

In [30]:
vertices(newton_polytope(f))

4-element SubObjectIterator{PointVector{QQFieldElem}}:
 [3, 4]
 [2, 4]
 [1, 3]
 [0, 1]

Note that Oscar view the verteices as *rational* vectors! For technical reasons, we need these to be **integer vectors** later in the notebook. We take care of it by simply applying `Int` componentwise to each vertex.

<strong>Coding trick:</strong> A nice feature in Julia is that the period <code>.</code> syntax can be used to apply a function <strong>component-wise</strong> (this is called <em>broadcasting</em> in case you want to google for it).  

If `f` is a function and `L` is a list/vector, then `f.(L)` applies `f` to every entry of `L`. This is equivalent to `map(f,L)`. (For operations like `+` or `^`, the parentheses are left out.)

Note: The period is also used to call *fields* of an object if there are no parentheses afterwards. Such fields are typically data attached to the object that is used behind the scenes. For example, if `f` is a polynomial, then `f.parent` is the ring where it lives.

<div class="alert alert-block alert-info">

<strong>Small exercises:</strong> Let `T, t = polynomial_ring(QQ,"t")`. What output does the following lines of code give?

`t.^[1,2,3]`

`(x -> x^2).([t,t+1,t+2])`

`evaluate.([t^2+1,t-1],1)`

`evaluate.(t^2+1,[1//1,2//1,3//1])`

</div>

In [31]:
# Convert a rational vector to an integer vector
integer_vector = v -> Int.(Vector(v));

In [32]:
# Example
integer_vector([2//1,-3//1,0//1])

3-element Vector{Int64}:
  2
 -3
  0

In [33]:
# Get all vertices of the Newton polytope, converted to integer vectors
vertices_as_integer_vectors = f -> integer_vector.(vertices(newton_polytope(f)));

In [34]:
vertices_as_integer_vectors(f)

4-element Vector{Vector{Int64}}:
 [3, 4]
 [2, 4]
 [1, 3]
 [0, 1]

## Find a vertex $\alpha_0$ with a specified sign

In [35]:
# Create a list of all negative vertices
negative_vertices = f -> [alpha for alpha in vertices_as_integer_vectors(f) if coeff(f,alpha)<0];

# Create a list of all positive vertices
positive_vertices = f -> negative_vertices(-f);

In [36]:
positive_vertices(f)

3-element Vector{Vector{Int64}}:
 [3, 4]
 [2, 4]
 [0, 1]

In [10]:
negative_vertices(f)

1-element Vector{Vector{Int64}}:
 [1, 3]

In [11]:
# Let's pick the first element in the list of positive vertices
alpha0 = positive_vertices(f)[1]

2-element Vector{Int64}:
 3
 4

## Find an outer normal vector $v$ of $\alpha_0$

In [12]:
# Compute the otuer normal cone of a vertex a0 of a polytope P
# Use that the ordering of normal_fan(P) mathces vertices(P)
inner_normal_cone = function(P,alpha0)
    alpha0_index = findfirst(==(alpha0), vertices(P))
    NF = normal_fan(P)
    return maximal_cones(NF)[alpha0_index]
end;

In [13]:
# Test of the maximal_cones command
C = maximal_cones(normal_fan(newton_polytope(f)))

4-element SubObjectIterator{Cone{QQFieldElem}}:
 Polyhedral cone in ambient dimension 2
 Polyhedral cone in ambient dimension 2
 Polyhedral cone in ambient dimension 2
 Polyhedral cone in ambient dimension 2

In [14]:
# Test of the rays command
rays(C[4])

2-element SubObjectIterator{RayVector{QQFieldElem}}:
 [1, -1//2]
 [-1, 1]

In [15]:
# Scale rational vector to integer vector by multiplying by the least common divisor
clear_denominators_of_vector = v -> integer_vector(lcm(denominator.(v))*v);

# Comptue an interior point of a polyhedral cone
integer_interior_point = function(C::Cone)
    RMS = rays_modulo_lineality(C).rays_modulo_lineality #compute the rays modulo the lineality space
     v0 = sum(RMS) # compute an interior vector by symming the rays
     return clear_denominators_of_vector(v0) 
end;

In [16]:
integer_outer_normal_vector = (P,alpha0) -> -integer_interior_point(inner_normal_cone(P,alpha0));

In [17]:
v = integer_outer_normal_vector(newton_polytope(f),alpha0)

2-element Vector{Int64}:
 1
 0

## Create the path $\phi_v$

In [18]:
path_from_vector = function(v::Vector{Int})
    S, t = LaurentPolynomialRing(QQ, "t")
    return t.^v
end;

In [19]:
phiv = path_from_vector(v)

2-element Vector{AbstractAlgebra.Generic.LaurentPolyWrap{QQFieldElem, QQPolyRingElem, AbstractAlgebra.Generic.LaurentPolyWrapRing{QQFieldElem, QQPolyRing}}}:
 t
 1

## Create the composition $f_v = f\circ\phi_v$

In [20]:
fv = evaluate(f,phiv)

8*t^3 + t^2 - 4*t + 1

## Find a $t_0$ such that $f_v(t_0)$ attains the sign of $\alpha_0$

A simple way to do this is to just start with some initial guess of $t_0$, say $t_0=1$, and the increase $t_0$ step by step. Eventually the leading term should become dominant, such that $f(t_0)$ has the same sign as the leading coefficient (which by design has the same sign as $\alpha_0$).

In [21]:
# Increase t0 from 1 in increments of 1, until it's large enough for f(t0) to attain the sign of the leading term of f
make_univariate_polynomial_attain_leading_sign = function(p)
    t0 = QQ(1)  # the rational number 1  
    while sign(evaluate(p,t0))!=sign(leading_coefficient(p))
        t0 = t0+1
    end
    return t0
end;

In [22]:
t0 = make_univariate_polynomial_attain_leading_sign(fv)

1

### Alternative solution (find the roots of $f_v$):

We could alternatively try to find the roots of the Laurent polynomial $f_v\in \mathbb{Q}[t^\pm]$. If $f_v$ turns out to have positive real roots, and $\mu$ is the largest one, then for any $t_0>\mu$, it holds that $f_v(t_0)$ has the same sign as its leading coefficient. For instance, we can pick $t_0=\lceil\mu\rceil+1$.

**How to find the roots of $f_v$:**

One perfectly fine option is to do this with Maple; just copy and paste your polynomial, and use the `solve` command.

If you prefer to stay in Julia/Oscar, we can instead use the `roots` command. Two small subtleties to pay attention to:

- We need to convert `fv` from a Laurent polynomial in $\mathbb{Q}[t^\pm]$ to a usual polynomial in $\mathbb{Q}[t]$ by writing `fv.poly`; this command simply multiplies a Luaurent polynomial by the smallest monomial needed to make all exponents nonnegative. 

- We need to tell `roots` that we want to solve over the *real numbers* (and not just the rational numbers) by writing `ArbField(64)` as an input; this creates a "model" of $\mathbb{R}$ with interval and floating point arithmetic. We can also write `AcbField(64)` if we want all complex solutions.

In [23]:
list_of_roots = roots(fv.poly,ArbField(64))

1-element Vector{arb}:
 [-0.8674850621292914964 +/- 8.66e-20]

From this we can read off manually that the only (and therefore also biggest) root of $f_v$ has ceiling 0, so we can e.g. pick $t_0=1$ to ensure $f_v(t_0)>0$.

If we feel ambitious and want to do this in more automatic way, we can carry out the following steps:
- Find the maximum of the entires in `list_of_roots`.
- Compute the ceiling.
- Convert to an integer.
- Convert from integer to a rational number.
- Add 1.

In [24]:
t0 = QQ(Int(ceil(maximum(list_of_roots))))+1

1

In [25]:
# Check that we get the right sign
evaluate(fv,t0)

6

## Let $a=\phi_v(t_0)$

If we have done everything correctly, it will now hold that $f(a)$ has the sign of $\alpha_0$.

In [26]:
a = evaluate.(phiv,t0)

2-element Vector{QQFieldElem}:
 1
 1

In [27]:
# Check that we get the correct sign
evaluate(f,a)

6